In [12]:
from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm, tqdm_notebook
#from tqdm import tqdm
#from tqdm.notebook import tqdm ,tqdm_notebook

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
DATA_PATH = 'C:/Users/10694387/Downloads/archive/IMDB Dataset.csv'
if not Path(DATA_PATH).is_file():
    gdd.download_file_from_google_drive(
        file_id='1zfM5E6HvKIe7f3rEt1V2gBpw5QOSSKQz',
        dest_path=DATA_PATH,
    )

In [4]:

# View some example records
pd.read_csv(DATA_PATH).sample(5)

,review,sentiment
38865,Schieder delivers a semi-believable part as th...,negative
40106,"I went to see ""Quitting"" with high hopes, beca...",negative
8972,When a rocket from a government experiment on ...,negative
40675,No one should ever try to adapt a Tom Robbins ...,negative
1124,Having been interested in Akhenaton for many y...,positive


In [5]:
class Sequences(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path)
        self.vectorizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
        self.sequences = self.vectorizer.fit_transform(df.review.tolist())
        self.labels = df.sentiment.tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]
    
    def __len__(self):
        return self.sequences.shape

In [67]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

df = pd.read_csv(DATA_PATH)

print(df.__len__())

train_data, test_data = train_test_split(df, test_size=0.2) # make the random split reproducible

len(train_data), len(test_data)

50000


(40000, 10000)

In [57]:
dataset = Sequences(DATA_PATH)
train_loader = DataLoader(dataset, batch_size=4096)

print(dataset[5][0].shape)
#print(train_loader)

In [36]:
class BagOfWordsClassifier(nn.Module):
    def __init__(self, vocab_size, hidden1, hidden2):
        super(BagOfWordsClassifier, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
    
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [37]:
model = BagOfWordsClassifier(len(dataset.token2idx), 128, 64)
model

BagOfWordsClassifier(
  (fc1): Linear(in_features=2891, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [68]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.001)



   

In [72]:
model.train()
train_losses = []
for epoch in range(10):
    #progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    #for sequences in progress_bar:
    model.zero_grad()

    output = model(train_data)
    loss = criterion(output.squeeze(), labels)
        
    loss.backward()
              
    nn.utils.clip_grad_norm_(model.parameters(), 3)

    optimizer.step()
        
    #    progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
    losses.append(loss.item())
    total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    #tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

AttributeError: 'DataFrame' object has no attribute 'float'